Adapted from https://github.com/catalystneuro/movshon-lab-to-nwb/blob/main/tutorials/blackrock_nwb_conversion_detailed.ipynb

In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [2]:
import os

dir_name = r'/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw'
base_filename = 'axona_raw_5s'
filename = os.path.join(dir_name, base_filename)
print(filename)

/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/axona_raw_5s


In [3]:
!ls ../nwb-conversion-tools/nwb_conversion_tools -l

total 76
-rwxrwxrwx 1 steburg steburg    69 Apr  5 11:28 __init__.py
-rwxrwxrwx 1 steburg steburg  1404 Apr  5 11:28 auto_qc.py
-rwxrwxrwx 1 steburg steburg  1338 Apr  5 11:28 basedatainterface.py
-rwxrwxrwx 1 steburg steburg  2134 Apr  5 11:28 baseimagingextractorinterface.py
-rwxrwxrwx 1 steburg steburg  2775 Apr  5 11:28 baselfpextractorinterface.py
-rwxrwxrwx 1 steburg steburg  4629 Apr  5 11:28 baserecordingextractorinterface.py
-rwxrwxrwx 1 steburg steburg  2363 Apr  5 11:28 basesegmentationextractorinterface.py
-rwxrwxrwx 1 steburg steburg  3829 Apr  5 11:28 basesortingextractorinterface.py
-rwxrwxrwx 1 steburg steburg  4312 Apr  5 11:28 conversion_tools.py
drwxrwxrwx 1 steburg steburg  4096 Apr  5 11:28 datainterfaces
-rwxrwxrwx 1 steburg steburg  5100 Apr  5 11:28 json_schema_utils.py
-rwxrwxrwx 1 steburg steburg 10154 Apr  5 11:28 metafile.schema.json
-rwxrwxrwx 1 steburg steburg  6665 Apr  5 11:28 nwbconverter.py
drwxrwxrwx 1 steburg steburg  4096 Apr  5 11:28 schemas
-rwxrw

In [4]:
# Import modules

import random
import string
import pytz
from typing import Union, Optional
from pathlib import Path
import spikeextractors as se
from pynwb import NWBFile
import numpy as np

from nwb_conversion_tools.baserecordingextractorinterface import BaseRecordingExtractorInterface,  BaseDataInterface
from nwb_conversion_tools.basesortingextractorinterface import BaseSortingExtractorInterface
from nwb_conversion_tools.json_schema_utils import get_schema_from_method_signature, get_base_schema, fill_defaults
from nwb_conversion_tools.datainterfaces.interface_utils.brpylib import NsxFile
from nwb_conversion_tools import SpikeGLXRecordingInterface

In [5]:
# from basedatainterface.py
metadata_schema = get_base_schema(
    id_='metadata.schema.json',
    root=True,
    title='Metadata',
    description='Schema for the metadata',
    version="0.1.0",
)

In [6]:
metadata_schema

{'required': [],
 'properties': {},
 'type': 'object',
 'additionalProperties': False,
 '$schema': 'http://json-schema.org/draft-07/schema#',
 '$id': 'metadata.schema.json',
 'title': 'Metadata',
 'description': 'Schema for the metadata',
 'version': '0.1.0'}

In [7]:
glx = SpikeGLXRecordingInterface

In [8]:
glx.get_source_schema()

{'required': ['file_path'],
 'properties': {'file_path': {'type': 'string',
   'format': 'file',
   'description': 'Path to SpikeGLX file.'}},
 'type': 'object',
 'additionalProperties': True}

In [9]:
import inspect

inspect.signature(glx.__init__).parameters

mappingproxy({'self': <Parameter "self">,
              'file_path': <Parameter "file_path: Union[str, pathlib.Path, NoneType]">,
              'stub_test': <Parameter "stub_test: Union[bool, NoneType] = False">})

In [12]:
test = dict(
            Ecephys=dict(
                Device=[
                    dict(
                        description="Device for the NWB Conversion Tools tutorial."
                    )
                ],
                ElectrodeGroup=[
                    dict(
                        name="ElectrodeGroup",
                        description="Electrode group for the NWB Conversion Tools tutorial."
                    )
                ],
                Electrodes=[
                    dict(
                        name="group_name",
                        description="Custom ElectrodeGroup name for these electrodes.",
                        data=["ElectrodeGroup" for x in range(1)]
                    ),
                    dict(
                        name="custom_electrodes_column",
                        description="Custom column in the electrodes table for the NWB Conversion Tools tutorial.",
                        data=[x for x in range(1)]
                    )
                ],
                ElectricalSeries=dict(
                    name="ElectricalSeries",
                    description="Raw acquisition traces for the NWB Conversion Tools tutorial."
                )
            )
        )

In [14]:
test = {'required': ['file_path'],
 'properties': {'file_path': {'type': 'string',
   'format': 'file',
   'description': 'Path to SpikeGLX file.'}},
 'type': 'object',
 'additionalProperties': True}

In [15]:
test

{'required': ['file_path'],
 'properties': {'file_path': {'type': 'string',
   'format': 'file',
   'description': 'Path to SpikeGLX file.'}},
 'type': 'object',
 'additionalProperties': True}

In [42]:
!ls ../spikeextractors/spikeextractors -l

total 224
-rwxrwxrwx 1 steburg steburg    86 Mar 24 10:00 MANIFEST.in
-rwxrwxrwx 1 steburg steburg  1265 Mar 19 08:48 __init__.py
drwxrwxrwx 1 steburg steburg  4096 Apr  5 09:53 __pycache__
-rwxrwxrwx 1 steburg steburg 22834 Mar 19 08:52 baseextractor.py
-rwxrwxrwx 1 steburg steburg  9269 Mar 19 08:52 cacheextractors.py
drwxrwxrwx 1 steburg steburg  4096 Mar 24 10:00 example_datasets
-rwxrwxrwx 1 steburg steburg   105 Mar 19 08:48 exceptions.py
-rwxrwxrwx 1 steburg steburg 36890 Mar 19 08:52 extraction_tools.py
-rwxrwxrwx 1 steburg steburg  5952 Apr  1 17:10 extractorlist.py
drwxrwxrwx 1 steburg steburg  4096 Mar 22 09:47 extractors
-rwxrwxrwx 1 steburg steburg  6494 Mar 19 08:52 multirecordingchannelextractor.py
-rwxrwxrwx 1 steburg steburg  8390 Mar 19 08:52 multirecordingtimeextractor.py
-rwxrwxrwx 1 steburg steburg  5458 Mar 19 08:52 multisortingextractor.py
-rwxrwxrwx 1 steburg steburg 41357 Mar 19 08:52 recordingextractor.py
-rwxrwxrwx 1 steburg steburg 30642 Mar 19 08:52 sorting

In [45]:
from spikeextractors.extractors.neoextractors import AxonaRecordingExtractor

In [53]:
re = AxonaRecordingExtractor(filename=filename)
print('Number of channels:', re.get_num_channels())
print('Channel groups:', re.get_channel_groups())

Number of channels: 16
Channel groups: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


Parse .set file for metadata to include in metadata.

In [65]:
def parse_generic_header(filename, params):
    """
    Given a binary file with phrases and line breaks, enters the
    first word of a phrase as dictionary key and the following
    string (without linebreaks) as value. Returns the dictionary.
    
    INPUT
    filename (str): .set file path and name.
    params (list or set): parameter names to search for. 
    
    OUTPUT
    header (dict): dictionary with keys being the parameters that
                   were found & values being strings of the data.
                   
    EXAMPLE
    parse_generic_header('myset_file.set', ['experimenter', 'trial_time'])
    """
    header = {}
    params = set(params)
    with open(filename, 'rb') as f:
        for bin_line in f:
            if b'data_start' in bin_line:
                break
            line = bin_line.decode('cp1252').replace('\r\n', '').replace('\r', '').strip()
            parts = line.split(' ')
            key = parts[0]
            if key in params:
                header[key] = ' '.join(parts[1:])
            
    return header

In [71]:
params_of_interest = [
    'trial_date',
    'trial_time',
    'experimenter', 
    'comments',
    'duration', 
    'sw_version',
    'tracker_version',
    'stim_version',
    'audio_version',
    'pretrigSamps',
    'spikeLockout'
]

In [72]:
set_file = filename+'.set'

In [73]:
parse_generic_header(set_file, params_of_interest)

{'trial_date': 'Sunday, 4 Oct 2020',
 'trial_time': '11:07:07',
 'experimenter': 'Abid',
 'comments': '',
 'duration': '600.00625',
 'sw_version': '1.2.2.16',
 'tracker_version': '0',
 'stim_version': '1',
 'audio_version': '0',
 'pretrigSamps': '10',
 'spikeLockout': '40'}

In [74]:
class AxonaRecordingExtractorInterface(BaseRecordingExtractorInterface):
    """Primary data interface class for converting a AxonaRecordingExtractor."""

    RX = se.AxonaRecordingExtractor

    @classmethod
    def get_source_schema(cls):
        source_schema = {
            'required': ['file_path'],
            'properties': {
                'file_path': {
                    'type': 'string',
                    'format': 'file',
                    'description': 'Path to Axona files.'
                }
            },
            'type': 'object',
            'additionalProperties': True
        }
        return source_schema
       
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def get_metadata(self):
        """Auto-fill as much of the metadata as possible. Must comply with metadata schema."""
        metadata = super().get_metadata()
        
        header = parse_generic_header(set_file, params_of_interest)
        
        metadata['properties']['Ecephys'] = \
            dict(
                Device=[
                    dict(
                        name="Axona",
                        description="Axona DacqUSB, sw_version={}".format(header['sw_version']),
                        manufacturer="Axona"
                    )
                ],
                ElectrodeGroup=[
                    dict(
                        name="ElectrodeGroup",
                        description="Electrode group (e.g. grouped by tetrodes).",
                        location="",
                        device="Axona"
                    )
                ],
                Electrodes=[
                    dict(
                        name="group_name",
                        description="Custom ElectrodeGroup name for these electrodes.",
                        data=["ElectrodeGroup" for x in range(self.recording_extractor.get_num_channels())]
                    ),
                    dict(
                        name="custom_electrodes_column",
                        description="Custom column in the electrodes table for the NWB Conversion Tools tutorial.",
                        data=[x for x in range(self.recording_extractor.get_num_channels())]
                    )
                ],
                ElectricalSeries=dict(
                    name="ElectricalSeries",
                    description="Raw acquisition traces from Axona .bin file."
                )
            )
        return metadata

In [86]:
schema['properties']['Ecephys']['required']

['Device', 'ElectrodeGroup', 'ElectricalSeries']

In [89]:
schema['properties']['Ecephys']['properties']['Device']['required']

['name']

In [98]:
schema['properties']['Ecephys']['properties']['Device']['properties']

{'name': {'description': 'Device name', 'type': 'string'},
 'description': 'Some Device',
 'manufacturer': {'description': 'the name of the manufacturer of this device',
  'type': 'string'}}

In [97]:
schema['properties']['Ecephys']['properties']['Device']['properties']['name']['description'] = 'Device name'

In [92]:
schema['properties']['Ecephys']['properties']['Device']['properties']['description'] = 'Some Device'

In [ ]:
schema = {
  "required": [],
  "properties": {
    "Ecephys": {
      "required": [
        "Device",
        "ElectrodeGroup",
        "ElectricalSeries"
      ],
      "properties": {
        "ElectricalSeries": {
          "required": [
            "name"
          ],
          "properties": {
            "name": {
              "description": "The name of this TimeSeries dataset",
              "type": "string",
              "default": "ElectricalSeries"
            },
            "resolution": {
              "description": "The smallest meaningful difference (in specified unit) between values in data",
              "type": "number",
              "default": -1.0
            },
            "conversion": {
              "description": "Scalar to multiply each element in data to convert it to the specified unit",
              "type": "number",
              "default": 1.0
            },
            "starting_time": {
              "description": "The timestamp of the first sample",
              "type": "number"
            },
            "rate": {
              "description": "Sampling rate in Hz",
              "type": "number"
            },
            "comments": {
              "description": "Human-readable comments about this TimeSeries dataset",
              "type": "string",
              "default": "no comments"
            },
            "description": {
              "description": "Description of this TimeSeries dataset",
              "type": "string",
              "default": "Raw acquisition traces for the NWB Conversion Tools tutorial."
            }
          },
          "type": "object",
          "additionalProperties": False,
          "tag": "pynwb.ecephys.ElectricalSeries"
        }
      },
      "type": "object",
      "additionalProperties": False,
      "tag": "Ecephys"
    }
  },
  "type": "object",
  "additionalProperties": False,
  "$schema": "http://json-schema.org/draft-07/schema#",
  "$id": "metadata.schema.json",
  "title": "Metadata",
  "description": "Schema for the metadata",
  "version": "0.1.0"
}

In [81]:
schema = {
  "required": [],
  "properties": {
    "Ecephys": {
      "required": [
        "Device",
        "ElectrodeGroup",
        "ElectricalSeries"
      ],
      "properties": {
        "Device": {
          "required": [
            "name"
          ],
          "properties": {
            "name": {
              "description": "the name of this device",
              "type": "string"
            },
            "description": {
              "description": "Description of the device (e.g., model, firmware version, processing software version, etc.)",
              "type": "string"
            },
            "manufacturer": {
              "description": "the name of the manufacturer of this device",
              "type": "string"
            }
          },
          "type": "object",
          "additionalProperties": False,
          "tag": "pynwb.device.Device"
        },
        "ElectrodeGroup": {
          "required": [
            "name",
            "description",
            "location",
            "device"
          ],
          "properties": {
            "name": {
              "description": "the name of this electrode",
              "type": "string"
            },
            "description": {
              "description": "description of this electrode group",
              "type": "string"
            },
            "location": {
              "description": "description of location of this electrode group",
              "type": "string"
            },
            "device": {
              "description": "the device that was used to record from this electrode group",
              "type": "string",
              "target": "pynwb.device.Device"
            }
          },
          "type": "object",
          "additionalProperties": False,
          "tag": "pynwb.ecephys.ElectrodeGroup"
        },
        "ElectricalSeries": {
          "required": [
            "name"
          ],
          "properties": {
            "name": {
              "description": "The name of this TimeSeries dataset",
              "type": "string",
              "default": "ElectricalSeries"
            },
            "resolution": {
              "description": "The smallest meaningful difference (in specified unit) between values in data",
              "type": "number",
              "default": -1.0
            },
            "conversion": {
              "description": "Scalar to multiply each element in data to convert it to the specified unit",
              "type": "number",
              "default": 1.0
            },
            "starting_time": {
              "description": "The timestamp of the first sample",
              "type": "number"
            },
            "rate": {
              "description": "Sampling rate in Hz",
              "type": "number"
            },
            "comments": {
              "description": "Human-readable comments about this TimeSeries dataset",
              "type": "string",
              "default": "no comments"
            },
            "description": {
              "description": "Description of this TimeSeries dataset",
              "type": "string",
              "default": "Raw acquisition traces for the NWB Conversion Tools tutorial."
            }
          },
          "type": "object",
          "additionalProperties": False,
          "tag": "pynwb.ecephys.ElectricalSeries"
        }
      },
      "type": "object",
      "additionalProperties": False,
      "tag": "Ecephys"
    }
  },
  "type": "object",
  "additionalProperties": False,
  "$schema": "http://json-schema.org/draft-07/schema#",
  "$id": "metadata.schema.json",
  "title": "Metadata",
  "description": "Schema for the metadata",
  "version": "0.1.0"
}

In [84]:
schema['properties']['Ecephys']

{'required': ['Device', 'ElectrodeGroup', 'ElectricalSeries'],
 'properties': {'Device': {'required': ['name'],
   'properties': {'name': {'description': 'the name of this device',
     'type': 'string'},
    'description': {'description': 'Description of the device (e.g., model, firmware version, processing software version, etc.)',
     'type': 'string'},
    'manufacturer': {'description': 'the name of the manufacturer of this device',
     'type': 'string'}},
   'type': 'object',
   'additionalProperties': False,
   'tag': 'pynwb.device.Device'},
  'ElectrodeGroup': {'required': ['name', 'description', 'location', 'device'],
   'properties': {'name': {'description': 'the name of this electrode',
     'type': 'string'},
    'description': {'description': 'description of this electrode group',
     'type': 'string'},
    'location': {'description': 'description of location of this electrode group',
     'type': 'string'},
    'device': {'description': 'the device that was used to rec

In [75]:
RX = AxonaRecordingExtractorInterface(filename=filename)

In [76]:
base_metadata_schema = RX.get_metadata_schema()

In [77]:
import json
print(json.dumps(base_metadata_schema, indent=2))

{
  "required": [],
  "properties": {
    "Ecephys": {
      "required": [
        "Device",
        "ElectrodeGroup",
        "ElectricalSeries"
      ],
      "properties": {
        "Device": {
          "required": [
            "name"
          ],
          "properties": {
            "name": {
              "description": "the name of this device",
              "type": "string"
            },
            "description": {
              "description": "Description of the device (e.g., model, firmware version, processing software version, etc.)",
              "type": "string"
            },
            "manufacturer": {
              "description": "the name of the manufacturer of this device",
              "type": "string"
            }
          },
          "type": "object",
          "additionalProperties": false,
          "tag": "pynwb.device.Device"
        },
        "ElectrodeGroup": {
          "required": [
            "name",
            "description",
            "lo

In [ ]:
# Example for metadata_schema from 
# https://github.com/catalystneuro/nwb-conversion-tools/blob/master/documentation/conversion_tools_structure.md#step-by-step-operations
{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "$id": "metafile.schema.json",
  "title": "Metadata",
  "description": "Schema for the metadata",
  "version": "0.1.0",
  "type": "object",
  "required": ["NWBFile"],
  "additionalProperties": false,
  "properties": {
    "NWBFile": {
      "type": "object",
      "additionalProperties": false,
      "tag": "pynwb.file.NWBFile",
      "required": ["session_description", "identifier", "session_start_time"],
      "properties": {
        "session_description": {
          "type": "string",
          "format": "long",
          "description": "a description of the session where this data was generated"
        },
        "identifier": {
          "type": "string",
          "description": "a unique text identifier for the file"
        },
        "session_start_time": {
          "type": "string",
          "description": "the start date and time of the recording session",
          "format": "date-time"
        }
      }
    },
    "Ecephys": {
      "type": "object",
      "title": "Ecephys",
      "required": [],
      "properties": {
        "Device": {"$ref": "#/definitions/Device"},
        "ElectricalSeries_raw": {"$ref": "#/definitions/ElectricalSeries"},
        "ElectricalSeries_processed": {"$ref": "#/definitions/ElectricalSeries"},
        "ElectrodeGroup": {"$ref": "#/definitions/ElectrodeGroup"}
      }
    }
  }
}

In [ ]:
NeuroscopeRecordingInterface

In [13]:
RXI = AxonaRecordingExtractorInterface(filename=filename)

In [14]:
RXI.source_data

{'filename': '/mnt/d/freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/axona_raw_5s'}

In [15]:
RXI.get_source_schema()

{'required': ['file_path'],
 'properties': {'file_path': {'type': 'string',
   'format': 'file',
   'description': 'Path to Axona files.'}},
 'type': 'object',
 'additionalProperties': True}

In [46]:
"""Authors: Luiz Tauffer"""
import random
import string
import pytz
from typing import Union, Optional
from pathlib import Path
import spikeextractors as se
from pynwb import NWBFile

from nwb_conversion_tools.baserecordingextractorinterface import BaseRecordingExtractorInterface
from nwb_conversion_tools.basesortingextractorinterface import BaseSortingExtractorInterface
from ..json_schema_utils import get_schema_from_method_signature
from .interface_utils.brpylib import NsxFile

PathType = Union[str, Path, None]


class BlackrockRecordingExtractorInterface(BaseRecordingExtractorInterface):
    """Primary data interface class for converting a BlackrockRecordingExtractor."""

    RX = se.BlackrockRecordingExtractor

    @classmethod
    def get_source_schema(cls):
        """Compile input schema for the RecordingExtractor."""
        metadata_schema = get_schema_from_method_signature(
            class_method=cls.RX.__init__,
            exclude=['block_index', 'seg_index']
        )
        metadata_schema['additionalProperties'] = True
        metadata_schema['properties']['filename']['format'] = 'file'
        metadata_schema['properties']['filename']['description'] = 'Path to Blackrock file.'
        return metadata_schema
    
    def __init__(self, filename: PathType, nsx_to_load: Optional[int] = None):
        super().__init__(filename=filename, nsx_to_load=nsx_to_load)

    def get_metadata(self):
        """Auto-fill as much of the metadata as possible. Must comply with metadata schema."""
        metadata = super().get_metadata()

        # Open file and extract headers
        set_file = NsxFile(datafile=self.source_data['filename'])
        session_start_time = nsx_file.basic_header['TimeOrigin']
        session_start_time_tzaware = pytz.timezone('EST').localize(session_start_time)
        comment = nsx_file.basic_header['Comment']

        metadata['NWBFile'] = dict(
            session_start_time=session_start_time_tzaware,
            session_description=comment,
            identifier=''.join(random.choices(string.ascii_uppercase + string.digits, k=12))
        )

        metadata['Ecephys'] = dict(
            Device=[dict(
                name='Device_ecephys',
                description='no description'
            )],
            ElectrodeGroup=[],
        )

        if self.source_data['nsx_to_load'] != 6:
            metadata['Ecephys']['LFPElectricalSeries'] = dict()
        else:
            metadata['Ecephys']['ElectricalSeries'] = dict()

        return metadata

    def run_conversion(self, nwbfile: NWBFile, metadata: dict = None, use_times: bool = False, 
                       write_as_lfp: bool = False, save_path: PathType = None, overwrite: bool = False, 
                       stub_test: bool = False):
        """
        Primary function for converting recording extractor data to nwb.
        Parameters
        ----------
        nwbfile: NWBFile
            nwb file to which the recording information is to be added
        metadata: dict
            metadata info for constructing the nwb file (optional).
            Should be of the format
                metadata['Ecephys']['ElectricalSeries'] = {'name': my_name,
                                                           'description': my_description}
        use_times: bool
            If True, the timestamps are saved to the nwb file using recording.frame_to_time(). If False (default),
            the sampling rate is used.
        write_as_lfp: bool (optional, defaults to False)
            If True, writes the traces under a processing LFP module in the NWBFile instead of acquisition.
        save_path: PathType
            Required if an nwbfile is not passed. Must be the path to the nwbfile
            being appended, otherwise one is created and written.
        overwrite: bool
            If using save_path, whether or not to overwrite the NWBFile if it already exists.
        stub_test: bool, optional (default False)
            If True, will truncate the data to run the conversion faster and take up less memory.
        """
        if self.source_data['nsx_to_load'] != 6:
            write_as_lfp = True

        super().run_conversion(
            nwbfile=nwbfile, 
            metadata=metadata, 
            use_times=use_times, 
            write_as_lfp=write_as_lfp,
            save_path=save_path, 
            overwrite=overwrite,
            stub_test=stub_test
        )


class BlackrockSortingExtractorInterface(BaseSortingExtractorInterface):
    """Primary data interface class for converting Blackrock spiking data."""

    SX = se.BlackrockSortingExtractor

    @classmethod
    def get_source_schema(cls):
        """Compile input schema for the RecordingExtractor."""
        metadata_schema = get_schema_from_method_signature(
            class_method=cls.SX.__init__,
            exclude=['block_index', 'seg_index', 'nsx_to_load']
        )
        metadata_schema['additionalProperties'] = True
        metadata_schema['properties']['filename']['format'] = 'file'
        metadata_schema['properties']['filename']['description'] = 'Path to Blackrock file.'
        return metadata_schema

    def __init__(self, filename: PathType, nsx_to_load: Optional[int] = None):
        super().__init__(filename=filename, nsx_to_load=nsx_to_load)

ImportError: cannot import name 'NeuroscopeRecordingInterface' from 'nwb_conversion_tools.basesortingextractorinterface' (/mnt/d/spikeinterface/hussaini-to-nwb/venv/lib/python3.8/site-packages/nwb_conversion_tools/basesortingextractorinterface.py)

# Movshon lab - Blackrock Converter
This tutorial follows the step-by-step guide for a [NWB Converter](https://github.com/catalystneuro/nwb-conversion-tools/blob/master/documentation/conversion_tools_structure.md#step-by-step-operations)

In [3]:
from hussaini_lab_to_nwb import HussainiNWBConverter
from pynwb import NWBFile, NWBHDF5IO
from nwbwidgets import nwb2widget
from pathlib import Path
import yaml
import pprint

## Step 1 - Converter.get_source_schema()

In [6]:
# Get source_schema
source_schema = HussainiNWBConverter.get_source_schema()
pprint.pprint(source_schema['properties'], width=120)

{'BlackrockRecordingExtractorInterface': {'additionalProperties': True,
                                          'properties': {'filename': {'description': 'Path to Blackrock file.',
                                                                      'format': 'file',
                                                                      'type': 'string'},
                                                         'nsx_to_load': {'type': 'number'}},
                                          'required': ['filename'],
                                          'type': 'object'},
 'BlackrockSortingExtractorInterface': {'additionalProperties': True,
                                        'properties': {'filename': {'description': 'Path to Blackrock file.',
                                                                    'format': 'file',
                                                                    'type': 'string'}},
                                        'required': ['filename'

## Step 2 - Get user-input source_data that complies to the returned full source_schema

In [4]:
# Source data
base_path = Path('/media/luiz/storage/taufferconsulting/client_ben/project_movshon/data/')
file_recording = str(base_path / 'XX_LE_textures_20191128_002.ns3')
file_sorting = str(base_path / 'XX_LE_textures_20191128_002.nev')

source_data = dict(
    BlackrockRecordingExtractorInterface=dict(
        filename=file_recording,
        nsx_to_load=3
    ),
    BlackrockSortingExtractorInterface=dict(
        filename=file_sorting,
        nsx_to_load=6
    )
)

pprint.pprint(source_data, width=120)

{'BlackrockRecordingExtractorInterface': {'filename': '/media/luiz/storage/taufferconsulting/client_ben/project_movshon/data/XX_LE_textures_20191128_002.ns3',
                                          'nsx_to_load': 3},
 'BlackrockSortingExtractorInterface': {'filename': '/media/luiz/storage/taufferconsulting/client_ben/project_movshon/data/XX_LE_textures_20191128_002.nev',
                                        'nsx_to_load': 6}}


## Step 3 - Instantiate Converter

In [79]:
# Initialize converter
converter = MovshonBlackrockNWBConverter(source_data=source_data)

print('Data interfaces for this converter:')
pprint.pprint(converter.data_interface_objects, width=120)

NameError: name 'MovshonBlackrockNWBConverter' is not defined

## Step 4 - Converter.get_metadata_schema()

In [78]:
# Get metadata_schema
metadata_schema = converter.get_metadata_schema()
pprint.pprint(metadata_schema, width=80)

NameError: name 'converter' is not defined

## Step 5 - Automatically fetches available metadata with Converter.get_metadata()

In [7]:
# Get metadata from source data
metadata = converter.get_metadata()
pprint.pprint(metadata, width=120)

{'Ecephys': {'Device': [{'description': 'no description', 'name': 'Device_ecephys'}],
             'ElectrodeGroup': [],
             'LFPElectricalSeries': {}},
 'NWBFile': {'identifier': 'JES40X1FWSHT',
             'session_description': '',
             'session_start_time': datetime.datetime(2019, 11, 28, 22, 35, 54, 729000, tzinfo=<StaticTzInfo 'EST'>)}}


## Step 6 - Get user-input metadata

In [8]:
metadata['NWBFile']['session_description'] = 'example conversion'
pprint.pprint(metadata, width=120)

{'Ecephys': {'Device': [{'description': 'no description', 'name': 'Device_ecephys'}],
             'ElectrodeGroup': [],
             'LFPElectricalSeries': {}},
 'NWBFile': {'identifier': 'JES40X1FWSHT',
             'session_description': 'example conversion',
             'session_start_time': datetime.datetime(2019, 11, 28, 22, 35, 54, 729000, tzinfo=<StaticTzInfo 'EST'>)}}


## Step 7 - Converter.get_conversion_options_schema()

In [9]:
# Not used for this converter

## Step 8 - Get user-input conversion options

In [10]:
# Not used for this converter

## Step 9 - Run conversion user filled metadata and conversion_options

In [11]:
output_file = 'out_example.nwb'

converter.run_conversion(
    metadata=metadata, 
    nwbfile_path=output_file, 
    save_to_file=True,
    conversion_options=None
)

/home/luiz/anaconda3/envs/env_klusta/lib/python3.7/site-packages/hdmf/common/table.py:415: UserWarning: An attribute 'name' already exists on DynamicTable 'electrodes' so this column cannot be accessed as an attribute, e.g., table.name; it can only be accessed using other methods, e.g., table['name'].
  warn(msg)


NWB file saved at out_example.nwb!


## Final 1 - Check NWB file

In [12]:
# load file
fname = 'out_example.nwb'
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)

root pynwb.file.NWBFile at 0x140281502197008
Fields:
  devices: {
    Device <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    0 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  file_create_date: [datetime.datetime(2021, 3, 5, 18, 47, 48, 213380, tzinfo=tzoffset(None, 3600))]
  identifier: JES40X1FWSHT
  processing: {
    ecephys <class 'pynwb.base.ProcessingModule'>
  }
  session_description: example conversion
  session_start_time: 2019-11-28 22:35:54.729000-05:00
  timestamps_reference_time: 2019-11-28 22:35:54.729000-05:00
  units: units <class 'pynwb.misc.Units'>



## Final 2 - Check NWB file with widgets

In [13]:
io = NWBHDF5IO(fname, 'r')
nwbfile = io.read()
nwb2widget(nwbfile)